In [1]:
# %load_ext cudf.pandas
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
sys.path.append('../src')
from utils import *
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import scipy.stats as stats
import scipy as sp
from scipy.stats import pearsonr,binom,linregress
from ast import literal_eval
import json
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import gaussian_kde
os.chdir("..")

In [3]:
file_paths = []

directory_path = f"output/dataframes/mindeye_extension_v6/subject1/statistics_df_mindeye_extension_v6_982.csv"
df = pd.read_csv(directory_path)
# df.insert(loc=1, column='Subject', value=1)
# df = column_string_to_list(df)
for subject in [2,5,7]:
    new_df = pd.read_csv(f"output/dataframes/mindeye_extension_v6/subject{subject}/statistics_df_mindeye_extension_v6_982.csv")
    # new_df.insert(loc=1, column='Subject', value=subject)
#     # new_df = column_string_to_list(new_df)
    df = pd.concat([df, new_df])


# print(df)
df.to_csv(f"output/dataframes/mindeye_extension_v6/all_subject_statistics.csv")
df_ss = df.loc[(df['Sample Indicator'] == 13) & (df['Subject'] == 1)]
df_me = df.loc[(df['Sample Indicator'] == 14) & (df['Subject'] == 1)]
df_ss.reset_index(drop=True, inplace=True)
df_me.reset_index(drop=True, inplace=True)

# Define the columns to be used for selecting the best row for each "ID"
columns_to_consider = [
    "CLIP Cosine",
    "Brain Correlation V1",
    "Brain Correlation V2",
    "Brain Correlation V3",
    "Brain Correlation V4",
    "Brain Correlation Higher Visual",
    "Brain Correlation NSD General",
    "AlexNet 2",
    "AlexNet 5",
    "AlexNet 7",
    "CLIP Two-way",
    "Inception V3"
] 

# Rank each of the relevant columns across the filtered DataFrame and sum these ranks for each row
df_ss_rank = df_ss[columns_to_consider].rank(ascending=False, method='min', na_option='keep')
df_ss['Rank_Sum'] = df_ss_rank.sum(axis=1)

df_me_rank = df_me[columns_to_consider].rank(ascending=False, method='min', na_option='keep')
df_me['Rank_Sum'] = df_me_rank.sum(axis=1)
df.head(100)
df_ss['Rank_Difference'] = df_ss['Rank_Sum'] - df_me['Rank_Sum']

df_ss['Rank_Sum_normalized'] = (df_ss['Rank_Sum'] - df_ss['Rank_Sum'].min()) / (df_ss['Rank_Sum'].max() - df_ss['Rank_Sum'].min())
df_ss['Rank_Difference_normalized'] = (df_ss['Rank_Difference'] - df_ss['Rank_Difference'].min()) / (df_ss['Rank_Difference'].max() - df_ss['Rank_Difference'].min())

# Create a combined column
df_ss['Weighted_Rank'] = (df_ss['Rank_Sum_normalized'] + df_ss['Rank_Difference_normalized']) / 2

# Find the row with the lowest sum of ranks for each "ID"
best_rows_filtered = df_ss.sort_values(by='Weighted_Rank', ascending=True)

# Generate file paths for the images corresponding to the rows after filtering
image_file_paths = [
    f"output/second_sight_paper/secondsight/subject{int(row['Subject'])}/{int(row['ID'])}/0.png"
    for index, row in best_rows_filtered.iterrows()
]

mindeye_file_paths = [
    f"output/second_sight_paper/mindeye/subject{int(row['Subject'])}/{int(row['ID'])}/0.png"
    for index, row in best_rows_filtered.iterrows()
]

# Generate file paths for the ground truth images corresponding to the filtered rows
ground_truth_file_paths = [
    f"output/second_sight_paper/secondsight/subject{int(row['Subject'])}/{int(row['ID'])}/ground_truth.png"
    for index, row in best_rows_filtered.iterrows()
]
for image in range(6):
    file_paths = []
    file_paths += ground_truth_file_paths[image*5:image*5+5]
    file_paths += image_file_paths[image*5:image*5+5]
    file_paths += mindeye_file_paths[image*5:image*5+5]
    print("FILES: ", ground_truth_file_paths[image*5:image*5+5])
    continue

    captions = ["Ground Truth", "Second Sight", "MindEye"]
    images = []
    for file in file_paths:
        images.append(Image.open(file))

    title =  f"Reconstructions for seen natural stimuli"
        
        
    tiled_figure = format_tiled_figure(
        images=images,
        captions=captions, 
        rows = 3, 
        cols = 5,
        red_line_index=0,  # Replace with the index of the column/row to highlight red
        buffer=20, # The buffer space in pixels between images.
        mode=3, # The mode of how captions and titles are used.
        title = title
        )
    tiled_figure.save(f"output/second_sight_paper/image_grids/SS_{image}_dif_top.png")

NameError: name 'pd' is not defined

In [ ]:
file_paths = []
# mode = "vision"
mode = "imagery"
stim = "simple"
for method in ["secondsight", "mindeye", "braindiffuser", "tagaki"]:
    
    directory_path = f"output/mental_imagery_paper/{mode}/{method}/subject1_statistics_12.csv"
    df = pd.read_csv(directory_path)
    # df.insert(loc=1, column='Subject', value=1)
    # df = column_string_to_list(df)
    for subject in [2,5,7]:
        new_df = pd.read_csv(f"output/mental_imagery_paper/{mode}/{method}/subject{subject}_statistics_12.csv")
        # new_df.insert(loc=1, column='Subject', value=subject)
    #     # new_df = column_string_to_list(new_df)
        df = pd.concat([df, new_df])

    # df.head(100)
    # print(df)
    df.to_csv(f"output/mental_imagery_paper/{mode}/{method}/all_subject_statistics.csv")

    df.reset_index(drop=True, inplace=True)
    # Define the columns to be used for selecting the best row for each "ID"
    columns_to_consider = [
        "CLIP Cosine",
        "Brain Correlation V1",
        "Brain Correlation V2",
        "Brain Correlation V3",
        "Brain Correlation V4",
        "Brain Correlation Higher Visual",
        "Brain Correlation NSD General",
        "AlexNet 2 1000",
        "AlexNet 5 1000",
        "AlexNet 7 1000",
        "CLIP Two-way 1000",
        "Inception V3 1000"
    ]

    # Filter the DataFrame for ID values less than or equal to 5 and "Sample Indicator" = 12
    filtered_df = df[(df['ID'] <= 5) & (df['Sample Indicator'] == 11)]

    # Rank each of the relevant columns across the filtered DataFrame and sum these ranks for each row
    ranked_df = filtered_df[columns_to_consider].rank(ascending=False, method='min', na_option='keep')
    filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)

    # Find the row with the lowest sum of ranks for each "ID"
    best_rows_indices_filtered = filtered_df.groupby('ID')['Rank_Sum'].idxmin()
    best_rows_filtered = filtered_df.loc[best_rows_indices_filtered].sort_values(by="ID")
    
    # Generate file paths for the images corresponding to the rows after filtering
    image_file_paths = [
    f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/low_level.png"
    for index, row in best_rows_filtered.iterrows()
    ]

    ground_truth_file_paths = [
        f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/ground_truth.png"
        for index, row in best_rows_filtered.iterrows()
    ]
    print(len(image_file_paths), len(ground_truth_file_paths))
    if method == "mindeye":
        file_paths += ground_truth_file_paths
    file_paths += image_file_paths
    
file_paths_t = transpose_flat_list(file_paths, 4, 6)
print(len(file_paths_t))
captions = ["Ground Truth", "MindEye", "Brain Diffuser", "Tagaki et al."]
# make_results_figure(f"output/mental_imagery_paper/figures/mi_{mode}_simple_low_level.png", file_paths_t, captions, rows=6, columns=4)
images = []
for file in file_paths:
    images.append(Image.open(file))
if mode == "vision":
    title =  f"Low level reconstructions for seen {stim} stimuli"
else:
    title =  f"Low level reconstructions for imagined {stim} stimuli"
tiled_figure = format_tiled_figure(
    images=images,
    captions=captions, 
    rows = 4, 
    cols = 6,
    red_line_index=0,  # Replace with the index of the column/row to highlight red
    buffer=20, # The buffer space in pixels between images.
    mode=3, # The mode of how captions and titles are used.
    title = title
    )
tiled_figure.save(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}_low_level.png")

0 0
6 6
6 6
6 6
24


/tmp/ipykernel_73404/3692248890.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_73404/3692248890.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_73404/3692248890.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [2]:
def make_paper_comp(indices):
    images = []
    path = "/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/output/second_sight_paper/"
    # for idx in indices:
    #     images.append(Image.open(f"{path}secondsight/subject1/{idx}/ground_truth.png"))
    for idx in indices:
        images.append(Image.open(f"{path}secondsight/subject1/{idx}/ground_truth.png"))
    for method in ["secondsight", "mindeye", "braindiffuser", "tagaki"]:
        for idx in indices:
            images.append(Image.open(f"{path}{method}/subject1/{idx}/0.png"))
            
    captions = ["Ground Truth", "MindEye + BOI", "MindEye", "Brain Diffuser", "Tagaki et al."]
    # title = f"Comparison of reconstruction methods on seen natural stimuli"
    tiled_figure = format_tiled_figure(
        images=images,
        captions=captions, 
        rows = 5, 
        cols = len(indices),
        red_line_index=0,  # Replace with the index of the column/row to highlight red
        buffer=20, # The buffer space in pixels between images.
        mode=3, # The mode of how captions and titles are used.
        # title = title
        )
    tiled_figure.save(f"output/second_sight_paper/image_grids/paper_comparison_horizontal.png")

make_paper_comp([22, 169, 325, 777, 874, 294, 554, 248, 960, 372])

In [6]:
def make_subject_comp(indices):
    images = []
    path = "/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/output/second_sight_paper/"
    # for idx in indices:
    #     images.append(Image.open(f"{path}secondsight/subject1/{idx}/ground_truth.png"))
    for idx in indices:
        images.append(Image.open(f"{path}secondsight/subject1/{idx}/ground_truth.png"))
    for subject in [1,2,5,7]:
        for idx in indices:
            images.append(Image.open(f"{path}secondsight/subject{subject}/{idx}/0.png"))
            
    captions = ["Ground Truth", "Subject 1", "Subject 2", "Subject 5", "Subject 7"]
    # title = f"Comparison of reconstruction methods on seen natural stimuli"
    tiled_figure = format_tiled_figure(
        images=images,
        captions=captions, 
        rows = 5, 
        cols = len(indices),
        red_line_index=0,  # Replace with the index of the column/row to highlight red
        buffer=20, # The buffer space in pixels between images.
        mode=3, # The mode of how captions and titles are used.
        # title = title
        )
    tiled_figure.save(f"output/second_sight_paper/image_grids/subject_comparison_horizontal.png")

make_subject_comp([90, 115, 122, 130, 159, 186, 488, 740, 44, 394])

In [4]:
def make_iteration_comp(indices):
    images = []
    blank = Image.new('RGB', (512, 512), color='white')
    path = "/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/output/mindeye_extension_v6/"
    # for idx in indices:
    #     images.append(Image.open(f"{path}secondsight/subject1/{idx}/ground_truth.png"))
    for idx in indices:
        images.append(Image.open(f"{path}subject1/{idx}/Ground Truth.png"))
        images.append(Image.open(f"{path}subject1/{idx}/MindEye.png"))
        for iteration in range(7):
            if os.path.exists(f"{path}subject1/{idx}/iter_{iteration}.png"):
                images.append(Image.open(f"{path}subject1/{idx}/iter_{iteration}.png"))
            else:
                for backiter in range(iteration, 0, -1):
                    if os.path.exists(f"{path}subject1/{idx}/iter_{backiter}.png"):
                        images.append(Image.open(f"{path}subject1/{idx}/iter_{backiter}.png"))
                        break

        images.append(Image.open(f"{path}subject1/{idx}/search_reconstruction.png"))
            
    captions = ["Ground Truth", "MindEye", "Iteration 1", "Iteration 2", "Iteration 3", "Iteration 4", "Iteration 5", "Iteration 6", "Iteration 7", "Final Reconstruction"]
    # title = f"Comparison of reconstruction methods on seen natural stimuli"
    tiled_figure = format_tiled_figure(
        images=images,
        captions=captions, 
        rows = len(indices), 
        cols = 10,
        red_line_index=0,  # Replace with the index of the column/row to highlight red
        buffer=20, # The buffer space in pixels between images.
        mode=2, # The mode of how captions and titles are used.
        # title = title
        )
    tiled_figure.save(f"output/second_sight_paper/image_grids/iteration_comparison_horizontal.png")

make_iteration_comp([234, 90, 130, 735, 882])#,
    # 319, 388, 427, 431, 437, 442, 451, 453, 488, 500, 501, 504, 506, 531, 547, 
    # 593, 609, 612, 616, 626, 730, 735, 765, 776, 779, 793, 830, 838, 848, 877,
    # 882, 955, 964])